# **Langauage Transcritption**

Managing voice records directly from google colab `using google transcription api` .

In [1]:
##Installation of pyaudio library on Linux system
##Note: Another format is used when laoding such library on a windows system
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio
import pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0


In [2]:
##Necessary importation for code running
!pip install webrtcvad 

     |████████████████████████████████| 66 kB 2.2 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72413 sha256=6475631ea3beb2d860369c463fa92cd04e48350c1092450585f4a5cc3bcb2c80
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
Successfully built webrtcvad


In [3]:
#Necessary importation for code running
!pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 12.1 MB/s 


In [4]:
#Necessary importation for code running
!pip install pydub

# **To DO (2)**

1. Complete the implementation of this section such that it can work locally omnon a computer.

In [5]:
##Codes to be used in the when the jupyter file will be converted into pythin file
"""
import wave
import os, sys
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf
import webrtcvad

vad = webrtcvad.Vad()
vad.set_mode(2)
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 32000
CHUNK = 960
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "file.wav"

audio = pyaudio.PyAudio()

# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
"""

'\nimport wave\nimport os, sys\nimport librosa\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport librosa.display\nimport tensorflow as tf\nimport webrtcvad\n\nvad = webrtcvad.Vad()\nvad.set_mode(2)\nFORMAT = pyaudio.paInt16\nCHANNELS = 1\nRATE = 32000\nCHUNK = 960\nRECORD_SECONDS = 3\nWAVE_OUTPUT_FILENAME = "file.wav"\n\naudio = pyaudio.PyAudio()\n\n# start Recording\nstream = audio.open(format=FORMAT, channels=CHANNELS,\n                rate=RATE, input=True,\n                frames_per_buffer=CHUNK)\n'

Managing audio records on google colab. This will change while working locally. 
A developer will have to add a section for such implementation locally for test to be done, for those who will use the codes locally.

In [6]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
from pydub import AudioSegment

wavfile = 'record.wav'
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

def verifyVoice(wavfile):
  """
  Function to verify the wave file record type
  Note: Not sure to be kept, seems not really necessary

  Input: None
  Output: Voice format
  """
  import fleep
  with open(wavfile, "rb") as file:
      info = fleep.get(file.read(128))

  print(info.extension)
  
  return info.extension[0]

def displayvoice(audio):
  import IPython.display as ipd
  ipd.display(ipd.Audio(audio))

def launchClientTranscript():
  """
  Function to convert voice directly.
  Note: This has been tested and works for the moment with english voice

  Input: None
  Output: voice format type, transcripted voice text
  """
  import speech_recognition as speech_recog
  rec = speech_recog.Recognizer() ##Google speech recognition API

  with open(wavfile, 'wb') as file:
       file.write(record()) ##saving the recorded file from audio
  flac_audio = AudioSegment.from_file(wavfile)
  flac_audio.export("record.wav", format="wav") ##Giving the recorded file an appropriate format wav format
  with speech_recog.AudioFile(wavfile) as source:
        audio_listened = rec.listen(source) ##Launching the API
  try:
    print("I think you said: \n" + rec.recognize_google(audio_listened))
    transcript = rec.recognize_google(audio_listened)
  except Exception as e:
    print(e)
  
  return verifyVoice(wavfile), transcript

In [7]:
launchClientTranscript()

Speak Now...


<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

# **TO DO (1)**

1. Test this google api transcription for voices in French if everything goes oni goes on proper.

2. Implement the google translate to convert French records into English transcripted from voice

In [ ]:
##Implement code

# testing the googletrans module to detect and translate text

In [35]:
# before everything, let's intall the module googletrans.
#I took this version because all the other version i use were not working propely
!pip install googletrans==4.0.0rc1

In [11]:
#now import the googletrans module
import googletrans 

In [12]:
#import also the class Translator that serve to use detect and translate methods
from googletrans import Translator

In [13]:
#we can view all the languages supported by googletrans 
print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [14]:
#creation of an object of the class Translator
translator = Translator()

In [15]:
#consider all of these text

text1 = 'bonjour tout le monde' #in french
text2 = 'good evening poeple'   #in english
text3 = 'wie geht es dir'     #in german

In [17]:
#let's use the method detect to know the language of these texts

print(translator.detect(text1))
print(translator.detect(text2))
print(translator.detect(text3))

Detected(lang=fr, confidence=None)
Detected(lang=en, confidence=None)
Detected(lang=de, confidence=None)


In [19]:
#let's use the method translate to translate each of these text into another language

# 1 - translation of the text1 from french to english
translation_of_text1 = translator.translate(text1,dest='en')

# 2 - translation of the text2 from english to german
translation_of_text2 = translator.translate(text1,dest='de')

# 3 - translation of the text3 from english to french
translation_of_text3 = translator.translate(text1,dest='fr')

In [22]:
print(translation_of_text1) 
print(translation_of_text2) 
print(translation_of_text3) 

Translated(src=fr, dest=en, text=Hello everybody, pronunciation=None, extra_data="{'confiden...")
Translated(src=fr, dest=de, text=Hallo zusammen, pronunciation=None, extra_data="{'confiden...")
Translated(src=fr, dest=fr, text=bonjour tout le monde, pronunciation=None, extra_data="{'confiden...")


In [34]:
#print just the text
print(f' |french| {text1}  => {translation_of_text1.text} |english| ') 
print(f'|english| {text2}  => {translation_of_text2.text}    |german|') 
print(f'|english| {text3}  => {translation_of_text3.text} |french|') 

 |french| bonjour tout le monde  => Hello everybody |english| 
|english| good evening poeple  => Hallo zusammen    |german|
|english| wie geht es dir  => bonjour tout le monde |french|
